In [ ]:
import torch
import torch.nn as nn
from torch.autograd.variable import *
import torch.optim as optim
import os
import numpy as np
import pandas as pd
import util
from __future__ import print_function
import setGPU
#os.environ['CUDA_VISIBLE_DEVICES']="6,7"

In [ ]:
#N = 100
save_path = '/nfshome/emoreno/IN/'
training_pivot = np.load(save_path + 'train_withSpectator_features_0.npy') #per jet constituents
training_pivot = np.swapaxes(training_pivot, 1, 2)
#training_0 = np.load(save_path + 'train_withSpectator_features_0.npy') #per jet constituents
#training_0 = np.swapaxes(training_0, 1, 2)
#training_1 = np.load(save_path + 'train_features_1.npy') #10 features of 100 inclusive (charged and neutral) particles
#training_1 = np.swapaxes(training_1, 1, 2)
training_2 = np.load(save_path + 'train_features_2.npy') #30 features of 60 charged particles
training_2 = np.swapaxes(training_2, 1, 2)
training_3 = np.load(save_path + 'train_features_3.npy') #14 features of 5 secondary vertices
training_3 = np.swapaxes(training_3, 1, 2)
target = np.load(save_path + 'train_truth_0.npy')
test_pivot = np.load(save_path + 'test_withSpectator_features_0.npy') #per jet constituents
test_pivot = np.swapaxes(test_pivot, 1, 2)
#test_0 = np.load(save_path + 'test_features_0.npy')
#test_0 = np.swapaxes(test_0, 1, 2)
#test_1 = np.load(save_path + 'test_features_1.npy')
#test_1 = np.swapaxes(test_1, 1, 2)
test_2 = np.load(save_path + 'test_features_2.npy')
test_2 = np.swapaxes(test_2, 1, 2)
test_3 = np.load(save_path + 'test_features_3.npy')
test_3 = np.swapaxes(test_3, 1, 2)
target_test = np.load(save_path + 'test_truth_0.npy')
params_0 = ['fj_jetNTracks',
          'fj_nSV',
          'fj_tau0_trackEtaRel_0',
          'fj_tau0_trackEtaRel_1',
          'fj_tau0_trackEtaRel_2',
          'fj_tau1_trackEtaRel_0',
          'fj_tau1_trackEtaRel_1',
          'fj_tau1_trackEtaRel_2',
          'fj_tau_flightDistance2dSig_0',
          'fj_tau_flightDistance2dSig_1',
          'fj_tau_vertexDeltaR_0',
          'fj_tau_vertexEnergyRatio_0',
          'fj_tau_vertexEnergyRatio_1',
          'fj_tau_vertexMass_0',
          'fj_tau_vertexMass_1',
          'fj_trackSip2dSigAboveBottom_0',
          'fj_trackSip2dSigAboveBottom_1',
          'fj_trackSip2dSigAboveCharm_0',
          'fj_trackSipdSig_0',
          'fj_trackSipdSig_0_0',
          'fj_trackSipdSig_0_1',
          'fj_trackSipdSig_1',
          'fj_trackSipdSig_1_0',
          'fj_trackSipdSig_1_1',
          'fj_trackSipdSig_2',
          'fj_trackSipdSig_3',
          'fj_z_ratio'
          ]

params_1 = ['pfcand_ptrel',
          'pfcand_erel',
          'pfcand_phirel',
          'pfcand_etarel',
          'pfcand_deltaR',
          'pfcand_puppiw',
          'pfcand_drminsv',
          'pfcand_drsubjet1',
          'pfcand_drsubjet2',
          'pfcand_hcalFrac'
         ]

params_2 = ['track_ptrel',     
          'track_erel',     
          'track_phirel',     
          'track_etarel',     
          'track_deltaR',
          'track_drminsv',     
          'track_drsubjet1',     
          'track_drsubjet2',
          'track_dz',     
          'track_dzsig',     
          'track_dxy',     
          'track_dxysig',     
          'track_normchi2',     
          'track_quality',     
          'track_dptdpt',     
          'track_detadeta',     
          'track_dphidphi',     
          'track_dxydxy',     
          'track_dzdz',     
          'track_dxydz',     
          'track_dphidxy',     
          'track_dlambdadz',     
          'trackBTag_EtaRel',     
          'trackBTag_PtRatio',     
          'trackBTag_PParRatio',     
          'trackBTag_Sip2dVal',     
          'trackBTag_Sip2dSig',     
          'trackBTag_Sip3dVal',     
          'trackBTag_Sip3dSig',     
          'trackBTag_JetDistVal'
         ]

params_3 = ['sv_ptrel',
          'sv_erel',
          'sv_phirel',
          'sv_etarel',
          'sv_deltaR',
          'sv_pt',
          'sv_mass',
          'sv_ntracks',
          'sv_normchi2',
          'sv_dxy',
          'sv_dxysig',
          'sv_d3d',
          'sv_d3dsig',
          'sv_costhetasvpv'
         ]

In [ ]:
#Convert two sets into two branch with one set in both and one set in only one (Use for this file)

training = training_2
test = test_2
params = params_2
training_sv = training_3
test_sv = test_3
params_sv = params_3
N = test.shape[2]

In [ ]:
def accuracy(predict, target):
    _, p_vals = torch.max(predict, 1)
    r = torch.sum(target == p_vals.squeeze(1)).data.numpy()[0]
    t = target.size()[0]
    return r * 1.0 / t

def stats(predict, target):
    _, p_vals = torch.max(predict, 1)
    t = target.cpu().data.numpy()
    p_vals = p_vals.squeeze(0).cpu().data.numpy()
    vals = np.unique(t)
    for i in vals:
        ind = np.where(t == i)
        pv = p_vals[ind]
        correct = sum(pv == t[ind])
        print("  Target %s: %s/%s = %s%%" % (i, correct, len(pv), correct * 100.0/len(pv)))
    print("Overall: %s/%s = %s%%" % (sum(p_vals == t), len(t), sum(p_vals == t) * 100.0/len(t)))
    return sum(p_vals == t) * 100.0/len(t)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
#import seaborn as sns
#sns.set()
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n + 1)

def predicted_histogram(data, 
                        target, 
                        labels = None, 
                        nbins = 10, 
                        out = None,
                        xlabel = None,
                        title = None
                       ):
    """@params:
        data = n x 1 array of parameter values
        target = n x categories array of predictions
    """
    target = preprocessing.normalize(target, norm = "l1")
    if labels == None:
        labels = ["" for i in range(target.shape[1])]
    #1 decide bins
    ma = np.amax(data) * 1.0
    mi = np.amin(data)
    bins = np.linspace(mi, ma, nbins)
    bin_size = bins[1] - bins[0]
    bin_locs = np.digitize(data, bins, right = True)
    #2 set up bin x category matrix
    #  Each M(bin, category) = Sum over particles with param in bin of category
    M = np.array([np.sum(target[np.where(bin_locs == i)], axis = 0) 
                  for i in range(nbins)])
    #3 plot each category/bin
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    bars = np.array([M[:, i] for i in range(M.shape[1])])
    cmap = get_cmap(len(bars), 'viridis')
    for i in range(len(bars)):
        ax.bar(bins, bars[i], 
               bottom = sum(bars[:i]), 
               color = cmap(i), 
               label = labels[i],
               width = bin_size
              )
    ax.set_xlabel(xlabel)
    ax.set_yticks([])
    ax.set_title(title)
    ax.legend()

def generate_control_plots():
    #global gnn
    len_params = len(params)
    path = '/nfshome/emoreno/IN/img/n-h-hb/'
    #os.makedirs(path)
    fr = 0
    b = 1000
    pred= None
    while fr< valv.shape[0]: #beginning splitting up valv into batches because memory runs out
        print ("Predicting from",fr)
        valv_1 = valv[fr:fr+b,...]
        p = gnn(valv_1.cuda())
        valv_1.cpu()
        p = p.cpu().data
        fr +=b
        if pred is None:
            pred = p
        else:
            pred = np.append(pred,p,axis=0)
        print (pred.shape) #end 

    d_target = np.array([util.get_list_from_num(i, length = n_targets) 
                             for i in val_targetv.cpu().data.numpy()])
    p_target = pred#.cpu().data.numpy()
    for i in range(len(params)):
        xlabel = params[i]
        labels = ["None", "H", "H + b"]
        data = np.mean(valv.data.numpy()[:, i, :], axis = 1)
        predicted_histogram(data, d_target, 
                            nbins = 50, labels = labels,
                            xlabel = xlabel, 
                            title = "Actual Distribution"
                           )
        plt.savefig(path + xlabel + "-actual.png", dpi = 200)
        predicted_histogram(data, p_target, 
                            nbins = 50, labels = labels,
                            xlabel = xlabel,
                            title = "Predicted Distribution"
                           )
        plt.savefig(path + xlabel + "-predicted.png", dpi = 200)
        plt.close("all")
    plt.show()


In [ ]:
import itertools
from sklearn import utils
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

def get_sample(training1, training2, training3, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = np.random.choice(ind, 200000)
    return training1[chosen_ind], training2[chosen_ind], training3[chosen_ind], target[chosen_ind]

def get_sample_train(training1, training2, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = ind
    #chosen_ind = np.random.choice(ind, 200000)
    return training1[chosen_ind], training2[chosen_ind], target[chosen_ind]

# Training Set
n_targets = target.shape[1]
samples = [get_sample(training, training_pivot,training_sv, target, i) for i in range(n_targets)]
trainings = [i[0] for i in samples]
trainings_pivot = [i[1] for i in samples]
trainings_sv = [i[2] for i in samples]
targets = [i[3] for i in samples]
big_training = np.concatenate(trainings)
big_training_pivot = np.concatenate(trainings_pivot)
big_training_sv = np.concatenate(trainings_sv)
big_target = np.concatenate(targets)
big_training, big_training_pivot, big_training_sv, big_target = utils.shuffle(big_training, big_training_pivot, big_training_sv, big_target)

val_split = 0.1
batch_size =128
n_epochs = 250

trainingv = Variable(torch.FloatTensor(big_training))
trainingv_pivot = Variable(torch.from_numpy(np.argmax(big_training_pivot, axis = 1)).long())
trainingv_sv = Variable(torch.FloatTensor(big_training_sv))
targetv = Variable(torch.from_numpy(np.argmax(big_target, axis = 1)).long()) 
trainingv, valv = torch.split(trainingv, int(trainingv.size()[0] * (1 - val_split)))
trainingv_pivot, valv_pivot = torch.split(trainingv_pivot, int(trainingv_pivot.size()[0] * (1 - val_split)))
trainingv_sv, valv_sv = torch.split(trainingv_sv, int(trainingv_sv.size()[0] * (1 - val_split)))
targetv, val_targetv = torch.split(targetv, int(targetv.size()[0] * (1 - val_split)))
samples_random = np.random.choice(range(len(trainingv)), valv.size()[0]/100)

#Test Set
n_targets_test = target_test.shape[1]
samples_test = [get_sample(test, test_pivot, test_sv, target_test, i) for i in range(n_targets_test)]
tests = [i[0] for i in samples_test]
tests_pivot = [i[1] for i in samples_test]
tests_sv = [i[2] for i in samples_test]
targets_tests = [i[3] for i in samples_test]
big_test = np.concatenate(tests)
big_test_pivot = np.concatenate(tests_pivot)
big_test_sv = np.concatenate(tests_sv)
big_target_test = np.concatenate(targets_tests)
big_test, big_test_pivot, big_test_sv, big_target_test = utils.shuffle(big_test, big_test_pivot, big_test_sv, big_target_test)

testv = Variable(torch.FloatTensor(big_test))
testv_pivot = Variable(torch.FloatTensor(big_test_pivot))
testv_sv = Variable(torch.FloatTensor(big_test_sv))
targetv_test = Variable(torch.from_numpy(np.argmax(big_target_test, axis = 1)).long())  
testv, valv_test = torch.split(testv, int(testv.size()[0] * (1 - val_split)))
testv_pivot, valv_test_pivot = torch.split(testv_pivot, int(testv_pivot.size()[0] * (1 - val_split)))
testv_sv, valv_test_sv = torch.split(testv_sv, int(testv_sv.size()[0] * (1 - val_split)))
targetv_test, val_targetv_test = torch.split(targetv_test, int(targetv_test.size()[0] * (1 - val_split)))




In [ ]:
softmax = torch.nn.Softmax()

class Rx(nn.Module):
    def __init__(self, ):
        super(Rx, self).__init__()
        self.dense1 = nn.Linear(2, 64).cuda()
        self.dense2 = nn.Linear(64, 64).cuda()
        self.dense3 = nn.Linear(64, 40).cuda()
        
    def forward(self, x, y, z):
        out = gnn(x, y, z)
        n = nn.functional.relu(self.dense1(out))
        n = nn.functional.relu(self.dense2(n))
        n = nn.functional.relu(self.dense2(n))
        n = nn.functional.relu(self.dense2(n))
        n = nn.functional.softmax(self.dense3(n), dim = 1)
        return n
        

loss_D = nn.BCELoss()
   
loss_R = nn.CrossEntropyLoss()
    
DfR = Rx()
#DfR.load_state_dict(torch.load('DfR_SV_tracks_0.4.0.torch_adversarial_adversary'))
opt_DfR = optim.SGD(DfR.parameters(), momentum = 0, lr = 0.0001)

In [ ]:
import itertools
from sklearn import utils
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.S = training_sv.shape[1]
        self.Nv = training_sv.shape[2]
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = 5
        self.Dx = 0
        self.Do = 6
        self.n_targets = n_targets
        self.assign_matrices()
        self.assign_matrices_SV()
        #self.switch = switch
        
        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden).cuda()
        self.fr1_sv = nn.Linear(self.S + self.P + self.Dr, hidden).cuda()
        self.fr2 = nn.Linear(hidden, hidden/2).cuda()
        self.fr3 = nn.Linear(hidden/2, self.De).cuda()
        self.fo1 = nn.Linear(self.P + self.Dx + (2 * self.De), hidden).cuda()
        self.fo2 = nn.Linear(hidden, hidden/2).cuda()
        self.fo3 = nn.Linear(hidden/2, self.Do).cuda()
        self.fc1 = nn.Linear(self.Do * self.N, hidden).cuda()
        self.fc2 = nn.Linear(hidden, hidden/2).cuda()
        self.fc3 = nn.Linear(hidden/2, self.n_targets).cuda()
        self.fc_fixed = nn.Linear(self.Do, self.n_targets).cuda()
        #self.gru = nn.GRU(input_size = self.Do, hidden_size = 20, bidirectional = False).cuda()
            
    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = (self.Rr).cuda()
        self.Rs = (self.Rs).cuda()
    
    def assign_matrices_SV(self):
        self.Rk = torch.zeros(self.N, self.Nr)
        self.Rv = torch.zeros(self.Nv, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.Nv)) if i[0]!=i[1]]
        for i, (k, v) in enumerate(receiver_sender_list):
            self.Rk[k, i] = 1
            self.Rv[v, i] = 1
        self.Rk = (self.Rk).cuda()
        self.Rv = (self.Rv).cuda()
        
    def forward(self, x, y, z):
        ###PF Candidate - PF Candidate###
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
        B = nn.functional.relu(self.fr2(B))
        E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        
        ####Secondary Vertex - PF Candidate### 
        Ork = self.tmul(x, self.Rk)
        Orv = self.tmul(y, self.Rv)
        B = torch.cat([Ork, Orv], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        B = nn.functional.relu(self.fr1_sv(B.view(-1, self.S + self.P + self.Dr)))
        B = nn.functional.relu(self.fr2(B))
        E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar_sv = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E

        ####Final output matrix###
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.cat([C, Ebar_sv], 1)
        del Ebar_sv
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + (2 * self.De))))
        C = nn.functional.relu(self.fo2(C))
        O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        #Taking the mean/sum of each column
        #N = torch.mean(O, dim=1)
        N = torch.sum(O, dim=1)
        del C
        ### Classification MLP ###
        #N = nn.functional.relu(self.fc1(O.view(-1, self.Do * self.N)))
        del O
        #N = nn.functional.relu(self.fc2(N))
        #N = nn.functional.relu(self.fc3(N))
        N = nn.functional.relu(self.fc_fixed(N))
        #P = np.array(N.data.cpu().numpy())
        #N = np.zeros((128, 1, 6))
        #for i in range(batch_size):
        #    N[i] = np.array(np.split(P[i], self.Do))
        #    N[1] = [P[i]]
        #N, hn = self.gru(torch.tensor(N).cuda())
        #print((N).shape)
        
        return N 
        
            
    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

# Neural Network
gnn = GraphNet(N, n_targets, params, 15)
#gnn.load_state_dict(torch.load('gnn_SV_tracks_0.4.0.torch_adversarial_classifier'))
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(gnn.parameters(), lr = 0.0001)

In [ ]:
loss_vals_training = np.zeros(n_epochs)
loss_validation_std = np.zeros(n_epochs)
loss_training_std = np.zeros(n_epochs)
loss_vals_validation = np.zeros(n_epochs)
acc_vals = np.zeros(n_epochs)
final_epoch = 0
lam = 100

# D Pretraining
for i in range(1):
    print("Epoch %s" % i)
    gnn = gnn.train()
    
    torch.cuda.empty_cache()
    final_epoch = i
    for j in range(0, trainingv.size()[0], batch_size):
        optimizer.zero_grad()
        out = gnn(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        l = loss(out, targetv[j:j + batch_size].cuda())
        l.backward()
        optimizer.step()
        loss_string = "Loss: %s" % "{0:.5f}".format(l.item())
        util.printProgressBar(j + batch_size, trainingv.size()[0], 
                              prefix = "%s [%s/%s] " % (loss_string, 
                                                       j + batch_size, 
                                                       trainingv.size()[0]),
                              length = 20)
    gnn = gnn.eval()
    # Validation Loss
    lst = []
    loss_val = []
    for j in range(0, valv.size()[0], batch_size):
        out = gnn(valv[j:j + batch_size].cuda(), valv_sv[j:j + batch_size].cuda(), valv_pivot[j:j + batch_size].cuda())
        lst.append(out.cpu().data.numpy())
        l_val = loss(out, val_targetv[j:j + batch_size].cuda())
        loss_val.append(l_val.item())
    l_val = np.mean(np.array(loss_val))
    predicted = (torch.FloatTensor(np.concatenate(lst))).to(device)
    print('\nValidation Loss: ', l_val)
    
    # Training Loss
    loss_training = []
    for j in samples_random:
        out = gnn(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        l_training = loss(out, targetv[j:j + batch_size].cuda())
        loss_training.append(l_training.item())
    l_training = np.mean(np.array(loss_training))
    print('Training Loss: ', l_training)
    gnn.train()
    
    acc_vals[i] = stats(predicted, val_targetv)

# R Pretraining
for i in range(1):
    print("Epoch %s" % i)
    #torch.cuda.empty_cache()
    final_epoch = i
    c = 1.0
    gnn = gnn.eval()
    
    for j in range(0, trainingv.size()[0], batch_size):
        opt_DfR.zero_grad()
        out = DfR(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        l_DfR = loss(out, trainingv_pivot[j:j + batch_size].cuda())
        l_DfR.backward()
        opt_DfR.step()
        loss_string = "Loss: %s" % "{0:.5f}".format(l_DfR.item())
        util.printProgressBar(j + batch_size, trainingv.size()[0], 
                              prefix = "%s [%s/%s] " % (loss_string, 
                                                       j + batch_size, 
                                                       trainingv.size()[0]),
                              length = 20)
    print('\n')
    
    
    lst = []
    loss_val = []
    for j in range(0, valv.size()[0], batch_size):
        out = DfR(valv[j:j + batch_size].cuda(), valv_sv[j:j + batch_size].cuda(), valv_pivot[j:j + batch_size].cuda())
        lst.append(out.cpu().data.numpy())
        l_val = loss_R(out, valv_pivot[j:j + batch_size].cuda())
        loss_val.append(l_val.item())
    l_val = np.mean(np.array(loss_val))
    predicted = (torch.FloatTensor(np.concatenate(lst))).to(device)
    print('\nValidation Loss: ', l_val)
    
    
# Adversarial Training    

for i in range(160):
    print("Epoch %s" % i)
    #torch.cuda.empty_cache()
    final_epoch = i
    DfR = DfR.eval()
    gnn = gnn.train()
    c = 1
    
    #train classifier
    for j in range(0, trainingv.size()[0], batch_size):
        optimizer.zero_grad()
        out = gnn(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        out_DfR = DfR(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        l = loss(out, targetv[j:j + batch_size].cuda())
        l_DfR = loss_R(out_DfR, trainingv_pivot[j:j + batch_size].cuda())
        l = (c * l) - (lam * l_DfR)
        l.backward()
        optimizer.step()
        loss_string = "Loss: %s" % "{0:.5f}".format(l.item())
        util.printProgressBar(j + batch_size, trainingv.size()[0], 
                              prefix = "%s [%s/%s] " % (loss_string, 
                                                       j + batch_size, 
                                                       trainingv.size()[0]),
                              length = 20)
    torch.save(gnn.state_dict(), 'gnn_SV_tracks_0.4.0.torch_adversarial_classifier_4')
    
    #train adversary
    for j in range(0, trainingv.size()[0], batch_size):    
        if lam > 0.0: 
            DfR = DfR.train()
            gnn = gnn.eval()
            opt_DfR.zero_grad()
            out_DfR = DfR(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
            l_DfR = c * loss_R(out_DfR, trainingv_pivot[j:j + batch_size].cuda())
            l_DfR.backward()
            opt_DfR.step()
            loss_string = "Loss: %s" % "{0:.5f}".format(l_DfR.item())
            util.printProgressBar(j + batch_size, trainingv.size()[0], 
                                  prefix = "%s [%s/%s] " % (loss_string, 
                                                           j + batch_size, 
                                                           trainingv.size()[0]),
                                  length = 20)
            
    torch.save(DfR.state_dict(), 'DfR_SV_tracks_0.4.0.torch_adversarial_adversary_4')
    
    gnn.eval()
    DfR.eval()
    # Validation Loss
    lst = []
    loss_val = []
    for j in range(0, valv.size()[0], batch_size):
        out = gnn(valv[j:j + batch_size].cuda(), valv_sv[j:j + batch_size].cuda(), valv_pivot[j:j + batch_size].cuda())
        lst.append(out.cpu().data.numpy())
        l_val = loss(out, val_targetv[j:j + batch_size].cuda())
        loss_val.append(l_val.item())
    l_val = np.mean(np.array(loss_val))
    predicted = (torch.FloatTensor(np.concatenate(lst))).to(device)
    print('\nValidation Loss: ', l_val)
    
    # Training Loss
    loss_training = []
    for j in samples_random:
        out = gnn(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda(), trainingv_pivot[j:j + batch_size].cuda())
        l_training = loss(out, targetv[j:j + batch_size].cuda())
        loss_training.append(l_training.item())
    l_training = np.mean(np.array(loss_training))
    print('Training Loss: ', l_training)
    
    acc_vals[20 + i] = stats(predicted, val_targetv)
    loss_vals_training[i] = l_training
    loss_vals_validation[i] = l_val
    loss_validation_std[i] = np.std(np.array(loss_val))
    loss_training_std[i] = np.std(np.array(loss_training))
    if all(loss_vals_validation[max(0, i - 5):i] > min(np.append(loss_vals_validation[0:max(0, i - 5)], 200))) and i > 5:
        print('Early Stopping...')
        print(loss_vals_training, '\n', np.diff(loss_vals_training))
        break
    print
    

In [ ]:
generate_control_plots()

In [ ]:
from IPython.display import Image, display
path = '/nfshome/emoreno/IN/img/n-h-hb/'
for xlabel in params:
    display(Image(filename=path + xlabel + '-actual.png'))
    display(Image(filename=path + xlabel + '-predicted.png'))

In [ ]:
# Generate Loss Plot
loss_vals_training = loss_vals_training[:(final_epoch)] 
loss_vals_validation = loss_vals_validation[:(final_epoch)] 
loss_validation_std = loss_validation_std[:(final_epoch)] 
loss_training_std = loss_training_std[:(final_epoch)] 
epochs = np.array(range(len(loss_vals_training)))
fig = plt.figure(figsize = (10,10))
ax1 = fig.add_subplot(111)
ax1.plot(epochs, loss_vals_training, label='training')
ax1.plot(epochs, loss_vals_validation, label='validation', color = 'green')
ax1.fill_between(epochs, loss_vals_validation - loss_validation_std/2, loss_vals_validation + loss_validation_std/2, color = 'lightgreen', label = 'Validation +/- 0.5 Std')
ax1.fill_between(epochs, loss_vals_training - loss_training_std/2, loss_vals_training + loss_training_std/2, color = 'lightblue', label = 'Training +/- 0.5 Std')
plt.legend(loc='upper right')
plt.title('Loss Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig('Loss_SV_tracks_adversarial')
plt.show()

In [ ]:
# Generate ROC Plot
from sklearn.metrics import roc_curve, roc_auc_score
softmax = torch.nn.Softmax(dim=1)

gnn.eval()
prediction = np.array([])
out = np.array([])
for j in range(0, testv.size()[0], batch_size):
    out_test = softmax(gnn(testv[j:j + batch_size].cuda(), testv_sv[j:j + batch_size].cuda(), testv_pivot[j:j + batch_size].cuda()))
    out_test = out_test.cpu().data.numpy()
   
    for i in range(len(out_test)):
        if (out_test[i][0] > out_test[i][1]): 
            prediction = np.append(prediction, out_test[i][0])
            out = np.append(out, 0)
        else: 
            prediction = np.append(prediction, out_test[i][1])
            out = np.append(out, 1)

for i in range(prediction.size): 
    if out[i] == 0: 
        prediction[i] = 1.0 - prediction[i]

   
fpr, tpr, thresholds = roc_curve(targetv_test.cpu().data.numpy(), prediction)
auc = roc_auc_score(targetv_test.cpu().data.numpy(), prediction)

fpr_DeepDoubleB = np.load('fpr_DeepDoubleB_LLF.npy')
tpr_DeepDoubleB = np.load('tpr_DeepDoubleB_LLF.npy')
dfpr_BDT = np.load('dfpr_BDT.npy')
dtpr_BDT = np.load('dtpr_BDT.npy')

plt.figure(figsize=(12,10))
lw = 2
plt.semilogy(tpr, fpr, color='darkorange',
         lw=lw, label='IN (area = %0.2f)' % auc)
#plt.plot(tpr_DeepDoubleB, fpr_DeepDoubleB, color='blue',
#         lw=lw, label='DeepDoubleB (area = 0.97)')
plt.plot(dtpr_BDT, dfpr_BDT, color='green',
         lw=lw, label='BDT (area = 0.914)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([10**-3, 1])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('ROC_adversarial')
plt.show()

In [ ]:
test_full = torch.FloatTensor(np.concatenate(np.array([test])))
test_sv_full = torch.FloatTensor(np.concatenate(np.array([test_sv])))
test_pivot_full = torch.FloatTensor(np.concatenate(np.array([test_pivot])))
prediction_test = np.array([])
gnn_out = np.array([])
for j in range(0, test_full.size()[0], batch_size):
    print(j)
    out_test = softmax(gnn(test_full[j:j + batch_size].cuda(), test_sv_full[j:j + batch_size].cuda(), test_pivot_full[j:j + batch_size].cuda()))
    out_test = out_test.cpu().data.numpy()
    for i in range(len(out_test)):
        if (out_test[i][0] > out_test[i][1]): 
            prediction_test = np.append(prediction_test, out_test[i][0])
            gnn_out = np.append(gnn_out, 0)
        else: 
            prediction_test = np.append(prediction_test, out_test[i][1])
            gnn_out = np.append(gnn_out, 1)

for i in range(prediction_test.size): 
    if gnn_out[i] == 0: 
        prediction_test[i] = 1.0 - prediction_test[i]
    
#np.save('out', out)
#np.save('prediction', prediction_test)

In [ ]:
train_full = torch.FloatTensor(np.concatenate(np.array([train])))
train_sv_full = torch.FloatTensor(np.concatenate(np.array([train_sv])))
prediction_train = np.array([])
gnn_out = np.array([])
for j in range(0, test_full.size()[0], batch_size * 10):
    print(j)
    out_test = softmax(gnn(test_full[j:j + batch_size * 10].cuda(), test_sv_full[j:j + batch_size * 10].cuda()))
    out_test = out_test.cpu().data.numpy()
    for i in range(len(out_test)):
        if (out_test[i][0] > out_test[i][1]): 
            prediction_test = np.append(prediction_test, out_test[i][0])
            gnn_out = np.append(gnn_out, 0)
        else: 
            prediction_test = np.append(prediction_test, out_test[i][1])
            gnn_out = np.append(gnn_out, 1)

for i in range(prediction_test.size): 
    if gnn_out[i] == 0: 
        prediction_test[i] = 1.0 - prediction_test[i]
    

In [ ]:
np.save('out', gnn_out)
np.save('prediction', prediction_test)
np.save('tpr', tpr)
np.save('fpr', fpr)
np.save('thresholds', thresholds)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(5,5), dpi = 200)
plt.plot(acc_vals[:20])
sns.set()
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
#plt.savefig("Accuracy_SV_tracks_2")
plt.show()